# <b> Deep Face Detection Model


### Installing Dependencies

In [ ]:
%pip install matplotlib albumentations --q
%pip install tensorflow-macos --q
%pip install labelme --q

#List of Dependencies that are installed
%pip list


In [43]:
import tensorflow as tf 
import os
import time
import uuid #Unique File Identifiers for the Images
import cv2
import json 
import numpy as numpy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import shutil

### Capture Images used for Data Segement and Augmentation

In [32]:
#Image Paths 
image_path = os.path.join('/Applications/Deep Learning/Face Detection/data', 'images')
number_of_images = 30

#If Directory Does not Exist
os.makedirs(image_path, exist_ok=True)

In [ ]:
#Open video source 
cap = cv2.VideoCapture(0)

while True:
    answer = input("Do you wish to capture Images(Y/N): ").lower()
    if answer != 'y':
        break
    for image_number in range(number_of_images):
        print('Collecting Image {}'.format(image_number))
        return_val, frame = cap.read() #Frame Number and Boolean Val for Success

        if not return_val:
            print('Image did not Successfully Capture')

        else:
            print('Picture Captured Successfully')
        img_name = os.path.join(image_path, f'{str(uuid.uuid1())}.jpg') #Unique UUID per image
        cv2.imwrite(img_name, frame) #Save Image
        cv2.imshow('frame', frame) #Display Image in Frame
        time.sleep(0.5)

        #Break out of Capturing Images if error occurs
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
   

cap.release()
cv2.destroyAllWindows()



### Annotate DataSet and Build Image Loading Function

In [4]:
#Create a Virtual Enviornment Just for LabelMe
''' 
conda create -n labelme python=3.10
conda activate labelme
pip install labelme
labelme
conda activate labelme (Use to Label the Images)
'''

zsh:1: command not found: labelme


In [11]:
#GPU Avaliable to be used?
print('GPU Avaliable:', tf.test.is_gpu_available('GPU'))

''' 
Load Images into TF Data Pipeline
Create Tensorflow of datasets, shuffle enables not inherent order)
'''
images = tf.data.Dataset.list_files('/Applications/Deep Learning/Face Detection/data/images/*.jpg', shuffle=False)


GPU Avaliable: False


In [12]:
''' 
Reads entire filestream of images and decodes JPEG file into 3D Tensor
Converts Images into Useable Tensor Datatype
'''
def load_image(img):
    byte_img = tf.io.read_file(img)
    image = tf.io.decode_jpeg(byte_img)
    return image


In [ ]:
#Map function that applies function to each item
images = images.map(load_image)

#Iterate through the images
images.as_numpy_iterator().next()

In [ ]:
#View Raw Images in Matplotlib
image_generator = images.batch(10).as_numpy_iterator()
print(type(image_generator))
next_batch = next(image_generator)


In [ ]:
#.next() contains the index and the image using .as_numpy_iterator()
while True:
    answer = input('Next Batch: ').lower()
    if answer != 'y':
        break
    else:
        plt.clf()
        next_batch = next(image_generator)
    
        fig, ax = plt.subplots(ncols=5, nrows=2, figsize=(10,10))
        for idx, image in enumerate(next_batch):
            row = idx//5
            col = idx%5
            ax[row,col].imshow(image)
            ax[row,col].axis('off')
        plt.show()
     

### Partition Unaugmented Data

* Note: It is possible to do the following task using split_folders (more standard in DL ). Higher level command and ensures that on any OS that the command works correctly. In addition splitting between the Data and the corresponding Label Class is also automatic

In [42]:
#Total Images is 382
print('Total Images in Dataset:', len([f for f in os.listdir(image_path)]))




#Train Dataset should be 70% of dataset
train_set, temp_set = train_test_split(total_images, test_size=0.3, random_state = 42)

#Validation and Test will be 15% each
validation_set, test_set = train_test_split(temp_set, test_size=0.5, random_state = 42)


print("Validation:", len(validation_set))
print("Test:", len(test_set))


Total Images in Dataset: 382
Train: 267
Validation: 57
Test: 58


In [48]:
image_path = '/Applications/Deep Learning/Face Detection/data/images'
base_path = '/Applications/Deep Learning/Face Detection/data'


#Put each Train Test Set into Respective Folder
train_dataset = os.path.join(base_path, 'train_set/images')
validation_dataset = os.path.join(base_path, 'validation_set/images')
test_dataset = os.path.join(base_path, 'test_set/images')


#Put Images in Respective Folder
for img in train_set:
     shutil.copy(os.path.join(image_path, img), os.path.join(train_dataset, img))
for img in validation_set:
     shutil.copy(os.path.join(image_path, img), os.path.join(validation_dataset, img))

for img in test_set:
     shutil.copy(os.path.join(image_path, img), os.path.join(test_dataset, img))

#Create Tensorflow dataset (Images)
train_dataset = tf.data.Dataset.list_files(os.path.join(train_dataset, '*.jpg'))
validation_dataset   = tf.data.Dataset.list_files(os.path.join(validation_dataset, '*.jpg'))
test_dataset  = tf.data.Dataset.list_files(os.path.join(test_dataset, '*.jpg'))


#Difficult Part is Matching Images well then you put the Corresponding Labels Manually 

In [ ]:
#This is not an efficient solution but it does the trick
label_path = '/Applications/Deep Learning/Face Detection/data/labels'
train_labels = file_names = ["00717294-eb18-11f0-93ba-c64476ca29a9", "00c7b1fe-eb18-11f0-93ba-c64476ca29a9", "011e76f6-eb18-11f0-93ba-c64476ca29a9", "0174d6f4-eb18-11f0-93ba-c64476ca29a9", "2cd419dc-eb17-11f0-93ba-c64476ca29a9", "2d287324-eb17-11f0-93ba-c64476ca29a9", "2d8081fe-eb17-11f0-93ba-c64476ca29a9", "2e2bf7b4-eb17-11f0-93ba-c64476ca29a9", "2edaaf20-eb17-11f0-93ba-c64476ca29a9", "2f2efe9a-eb17-11f0-93ba-c64476ca29a9", "2f873c04-eb17-11f0-93ba-c64476ca29a9", "30328532-eb17-11f0-93ba-c64476ca29a9", "308945fc-eb17-11f0-93ba-c64476ca29a9", "30dfb86a-eb17-11f0-93ba-c64476ca29a9", "3137cfdc-eb17-11f0-93ba-c64476ca29a9", "318ea8ca-eb17-11f0-93ba-c64476ca29a9", "31e4e1e0-eb17-11f0-93ba-c64476ca29a9", "323b67ea-eb17-11f0-93ba-c64476ca29a9", "32e71c20-eb17-11f0-93ba-c64476ca29a9", "333d6c2e-eb17-11f0-93ba-c64476ca29a9", "33954c00-eb17-11f0-93ba-c64476ca29a9", "33ea91f6-eb17-11f0-93ba-c64476ca29a9", "34978708-eb17-11f0-93ba-c64476ca29a9", "34ee40fc-eb17-11f0-93ba-c64476ca29a9", "35f19170-eb17-11f0-93ba-c64476ca29a9", "3648aef6-eb17-11f0-93ba-c64476ca29a9", "369e2034-eb17-11f0-93ba-c64476ca29a9", "4717bb46-eb17-11f0-93ba-c64476ca29a9", "476b60c0-eb17-11f0-93ba-c64476ca29a9", "47c24552-eb17-11f0-93ba-c64476ca29a9", "49c8fdd2-eb17-11f0-93ba-c64476ca29a9", "4a202062-eb17-11f0-93ba-c64476ca29a9", "4a761d96-eb17-11f0-93ba-c64476ca29a9", "4b22aca0-eb17-11f0-93ba-c64476ca29a9", "4b7a4758-eb17-11f0-93ba-c64476ca29a9", "4bd011d8-eb17-11f0-93ba-c64476ca29a9", "4c273120-eb17-11f0-93ba-c64476ca29a9", "4c7d470e-eb17-11f0-93ba-c64476ca29a9", "4cd3f0cc-eb17-11f0-93ba-c64476ca29a9", "4d2a28e8-eb17-11f0-93ba-c64476ca29a9", "4d810708-eb17-11f0-93ba-c64476ca29a9", "4dd77caa-eb17-11f0-93ba-c64476ca29a9", "4e842216-eb17-11f0-93ba-c64476ca29a9", "4edb56ee-eb17-11f0-93ba-c64476ca29a9", "4f308a10-eb17-11f0-93ba-c64476ca29a9", "4f88b7b2-eb17-11f0-93ba-c64476ca29a9", "4fde6a18-eb17-11f0-93ba-c64476ca29a9", "5035b138-eb17-11f0-93ba-c64476ca29a9", "508bdf18-eb17-11f0-93ba-c64476ca29a9", "5fd29d32-eb16-11f0-93ba-c64476ca29a9", "608dbee6-eb16-11f0-93ba-c64476ca29a9", "60e406ca-eb16-11f0-93ba-c64476ca29a9", "619bf3b6-eb16-11f0-93ba-c64476ca29a9", "624a4632-eb16-11f0-93ba-c64476ca29a9", "62a05c34-eb16-11f0-93ba-c64476ca29a9", "62f67a06-eb16-11f0-93ba-c64476ca29a9", "63f82fb2-eb16-11f0-93ba-c64476ca29a9", "644c3710-eb16-11f0-93ba-c64476ca29a9", "64a24380-eb16-11f0-93ba-c64476ca29a9", "64f965e8-eb16-11f0-93ba-c64476ca29a9", "654f8d1a-eb16-11f0-93ba-c64476ca29a9", "65a5b190-eb16-11f0-93ba-c64476ca29a9", "65fc70de-eb16-11f0-93ba-c64476ca29a9", "6b302374-eb17-11f0-93ba-c64476ca29a9", "6b898702-eb17-11f0-93ba-c64476ca29a9", "6be282d0-eb17-11f0-93ba-c64476ca29a9", "6c36b13e-eb17-11f0-93ba-c64476ca29a9", "6ce3be38-eb17-11f0-93ba-c64476ca29a9", "6d3a59aa-eb17-11f0-93ba-c64476ca29a9", "6de84538-eb17-11f0-93ba-c64476ca29a9", "6e3e6c74-eb17-11f0-93ba-c64476ca29a9", "6e9512cc-eb17-11f0-93ba-c64476ca29a9", "6eeb308a-eb17-11f0-93ba-c64476ca29a9", "6f97e2c6-eb17-11f0-93ba-c64476ca29a9", "709a6dce-eb17-11f0-93ba-c64476ca29a9", "70f242a6-eb17-11f0-93ba-c64476ca29a9", "71476d80-eb17-11f0-93ba-c64476ca29a9", "719f025c-eb17-11f0-93ba-c64476ca29a9", "724b4a26-eb17-11f0-93ba-c64476ca29a9", "72f941da-eb17-11f0-93ba-c64476ca29a9", "7350227a-eb17-11f0-93ba-c64476ca29a9", "73a63f34-eb17-11f0-93ba-c64476ca29a9", "74529838-eb17-11f0-93ba-c64476ca29a9", "74a9fde4-eb17-11f0-93ba-c64476ca29a9", "7500695e-eb17-11f0-93ba-c64476ca29a9", "779b3856-eb17-11f0-93ba-c64476ca29a9", "77f0387e-eb17-11f0-93ba-c64476ca29a9", "7846f5ec-eb17-11f0-93ba-c64476ca29a9", "794b745e-eb17-11f0-93ba-c64476ca29a9", "7a4f1b58-eb17-11f0-93ba-c64476ca29a9", "7aa41478-eb17-11f0-93ba-c64476ca29a9", "7afc04d0-eb17-11f0-93ba-c64476ca29a9", "7bfef96e-eb17-11f0-93ba-c64476ca29a9", "7c56c40a-eb17-11f0-93ba-c64476ca29a9", "7cad190e-eb17-11f0-93ba-c64476ca29a9", "7d58664c-eb17-11f0-93ba-c64476ca29a9", "7db0d5b6-eb17-11f0-93ba-c64476ca29a9", "7e06aee6-eb17-11f0-93ba-c64476ca29a9", "7eb29d0a-eb17-11f0-93ba-c64476ca29a9", "7f60b368-eb17-11f0-93ba-c64476ca29a9", "7fb639f0-eb17-11f0-93ba-c64476ca29a9", "805e0aa4-eb17-11f0-93ba-c64476ca29a9", "80b59fa8-eb17-11f0-93ba-c64476ca29a9", "810accf8-eb17-11f0-93ba-c64476ca29a9", "8162a950-eb17-11f0-93ba-c64476ca29a9", "8a49e3fc-eb18-11f0-93ba-c64476ca29a9", "8b14ccfc-eb18-11f0-93ba-c64476ca29a9", "8b6b28b8-eb18-11f0-93ba-c64476ca29a9", "8c171c90-eb18-11f0-93ba-c64476ca29a9", "8c6d5b3c-eb18-11f0-93ba-c64476ca29a9", "8cb7a616-eb17-11f0-93ba-c64476ca29a9", "8cc568c2-eb18-11f0-93ba-c64476ca29a9", "8d0d89f0-eb17-11f0-93ba-c64476ca29a9", "8d1b2ce4-eb18-11f0-93ba-c64476ca29a9", "8d65289a-eb17-11f0-93ba-c64476ca29a9", "8d70f368-eb18-11f0-93ba-c64476ca29a9", "8dbb3546-eb17-11f0-93ba-c64476ca29a9", "8dc75d2a-eb18-11f0-93ba-c64476ca29a9", "8e1e90c2-eb18-11f0-93ba-c64476ca29a9", "8e62f06a-eb17-11f0-93ba-c64476ca29a9", "8e74ada4-eb18-11f0-93ba-c64476ca29a9", "8eb8eef2-eb17-11f0-93ba-c64476ca29a9", "8ecb3f16-eb18-11f0-93ba-c64476ca29a9", "8f0e7f84-eb17-11f0-93ba-c64476ca29a9", "8f23390a-eb18-11f0-93ba-c64476ca29a9", "8f61676c-eb17-11f0-93ba-c64476ca29a9", "8f798170-eb18-11f0-93ba-c64476ca29a9", "8fcf5da2-eb18-11f0-93ba-c64476ca29a9", "900d8006-eb17-11f0-93ba-c64476ca29a9", "9026950e-eb18-11f0-93ba-c64476ca29a9", "90d3af28-eb18-11f0-93ba-c64476ca29a9", "9110e2f4-eb17-11f0-93ba-c64476ca29a9", "912a9810-eb18-11f0-93ba-c64476ca29a9", "9167dac8-eb17-11f0-93ba-c64476ca29a9", "91805d04-eb18-11f0-93ba-c64476ca29a9", "91bd5a8e-eb17-11f0-93ba-c64476ca29a9", "91d5d1c6-eb18-11f0-93ba-c64476ca29a9", "9215c76e-eb17-11f0-93ba-c64476ca29a9", "922c3dfe-eb18-11f0-93ba-c64476ca29a9", "9282db82-eb18-11f0-93ba-c64476ca29a9", "92c219e2-eb17-11f0-93ba-c64476ca29a9", "92d91bbe-eb18-11f0-93ba-c64476ca29a9", "93187d78-eb17-11f0-93ba-c64476ca29a9", "933177d2-eb18-11f0-93ba-c64476ca29a9", "93703b6c-eb17-11f0-93ba-c64476ca29a9", "9387f864-eb18-11f0-93ba-c64476ca29a9", "93c5b786-eb17-11f0-93ba-c64476ca29a9", "93dd96ac-eb18-11f0-93ba-c64476ca29a9", "941ccdbe-eb17-11f0-93ba-c64476ca29a9", "9434df98-eb18-11f0-93ba-c64476ca29a9", "94730378-eb17-11f0-93ba-c64476ca29a9", "95cd2b86-eb17-11f0-93ba-c64476ca29a9", "9623f6d2-eb17-11f0-93ba-c64476ca29a9", "9679d854-eb17-11f0-93ba-c64476ca29a9", "a23e1088-eb17-11f0-93ba-c64476ca29a9", "a2937f1e-eb17-11f0-93ba-c64476ca29a9", "a2e6a9d2-eb17-11f0-93ba-c64476ca29a9", "a33ba8c4-eb17-11f0-93ba-c64476ca29a9", "a39416ee-eb17-11f0-93ba-c64476ca29a9", "a3e8716c-eb17-11f0-93ba-c64476ca29a9", "a440c150-eb17-11f0-93ba-c64476ca29a9", "a4958988-eb17-11f0-93ba-c64476ca29a9", "a4ec86ac-eb17-11f0-93ba-c64476ca29a9", "a599c0b0-eb17-11f0-93ba-c64476ca29a9", "a5efc1ea-eb17-11f0-93ba-c64476ca29a9", "ace1b21a-eb17-11f0-93ba-c64476ca29a9", "ad8e5da8-eb17-11f0-93ba-c64476ca29a9", "ae3b5dc8-eb17-11f0-93ba-c64476ca29a9", "ae91c398-eb17-11f0-93ba-c64476ca29a9", "af3eece4-eb17-11f0-93ba-c64476ca29a9", "af953bf8-eb17-11f0-93ba-c64476ca29a9", "afeb7126-eb17-11f0-93ba-c64476ca29a9", "b042b698-eb17-11f0-93ba-c64476ca29a9", "b0981480-eb17-11f0-93ba-c64476ca29a9", "b0eee292-eb17-11f0-93ba-c64476ca29a9", "b145210c-eb17-11f0-93ba-c64476ca29a9", "b19c3e2e-eb17-11f0-93ba-c64476"]
test_labels = ["2dd6eca6-eb17-11f0-93ba-c64476ca29a9","2e8357de-eb17-11f0-93ba-c64476ca29a9","34410248-eb17-11f0-93ba-c64476ca29a9","354559c8-eb17-11f0-93ba-c64476ca29a9","48176e88-eb17-11f0-93ba-c64476ca29a9","4e2e0b56-eb17-11f0-93ba-c64476ca29a9","50e20938-eb17-11f0-93ba-c64476ca29a9","61f2bb7e-eb16-11f0-93ba-c64476ca29a9","634dbfa0-eb16-11f0-93ba-c64476ca29a9","6c8d3ce8-eb17-11f0-93ba-c64476ca29a9","6f421292-eb17-11f0-93ba-c64476ca29a9","7045498e-eb17-11f0-93ba-c64476ca29a9","72a24ec0-eb17-11f0-93ba-c64476ca29a9","78f5070e-eb17-11f0-93ba-c64476ca29a9","79f8417a-eb17-11f0-93ba-c64476ca29a9","7b5246a6-eb17-11f0-93ba-c64476ca29a9","7d03b6c4-eb17-11f0-93ba-c64476ca29a9","7f0926b6-eb17-11f0-93ba-c64476ca29a9","8ab2ceb2-eb18-11f0-93ba-c64476ca29a9","8fb7f726-eb17-11f0-93ba-c64476ca29a9","90ba15fa-eb17-11f0-93ba-c64476ca29a9","9576deac-eb17-11f0-93ba-c64476ca29a9","a1e8c57e-eb17-11f0-93ba-c64476ca29a9","a542fe60-eb17-11f0-93ba-c64476ca29a9","a6463c64-eb17-11f0-93ba-c64476ca29a9","ade4864c-eb17-11f0-93ba-c64476ca29a9","b2f6a41c-eb17-11f0-93ba-c64476ca29a9","b4fdd2d0-eb17-11f0-93ba-c64476ca29a9","ba1445ec-eb17-11f0-93ba-c64476ca29a9","bb165340-eb17-11f0-93ba-c64476ca29a9","bb6e9df2-eb17-11f0-93ba-c64476ca29a9","bc7055f6-eb17-11f0-93ba-c64476ca29a9","bd1e3842-eb17-11f0-93ba-c64476ca29a9","bd74318e-eb17-11f0-93ba-c64476ca29a9","dbba2a96-eb16-11f0-93ba-c64476ca29a9","de427f7a-eb16-11f0-93ba-c64476ca29a9","de9544da-eb16-11f0-93ba-c64476ca29a9","dfef42fe-eb16-11f0-93ba-c64476ca29a9","e3a58dc2-eb16-11f0-93ba-c64476ca29a9","e6095f62-eb16-11f0-93ba-c64476ca29a9","e6b56c08-eb16-11f0-93ba-c64476ca29a9","e7b8c6f4-eb16-11f0-93ba-c64476ca29a9","eabef95e-eb16-11f0-93ba-c64476ca29a9","ec198486-eb16-11f0-93ba-c64476ca29a9","ed1cd7c0-eb16-11f0-93ba-c64476ca29a9","ed72cfe0-eb16-11f0-93ba-c64476ca29a9","edc8bda6-eb16-11f0-93ba-c64476ca29a9","f17b7bdc-eb16-11f0-93ba-c64476ca29a9","f1e0ad08-eb17-11f0-93ba-c64476ca29a9","f28d7b14-eb17-11f0-93ba-c64476ca29a9","f4eb0854-eb17-11f0-93ba-c64476ca29a9","f597d6c4-eb17-11f0-93ba-c64476ca29a9","f748030e-eb17-11f0-93ba-c64476ca29a9","f8a26b22-eb17-11f0-93ba-c64476ca29a9","f8f96ff8-eb17-11f0-93ba-c64476ca29a9","f9a624d2-eb17-11f0-93ba-c64476ca29a9","fa5375ba-eb17-11f0-93ba-c64476ca29a9","ff747120-eb17-11f0-93ba-c64476ca29a9"]
validation_labels = ["001a711a-eb18-11f0-93ba-c64476ca29a9","2fdcd02e-eb17-11f0-93ba-c64476ca29a9","329177b6-eb17-11f0-93ba-c64476ca29a9","359af54a-eb17-11f0-93ba-c64476ca29a9","486fd3ca-eb17-11f0-93ba-c64476ca29a9","48c5e062-eb17-11f0-93ba-c64476ca29a9","491c5956-eb17-11f0-93ba-c64476ca29a9","4973332a-eb17-11f0-93ba-c64476ca29a9","4acc54ea-eb17-11f0-93ba-c64476ca29a9","60328cec-eb16-11f0-93ba-c64476ca29a9","613aaba6-eb16-11f0-93ba-c64476ca29a9","63a19f94-eb16-11f0-93ba-c64476ca29a9","6651a2ca-eb16-11f0-93ba-c64476ca29a9","6d8ff518-eb17-11f0-93ba-c64476ca29a9","6fef4b56-eb17-11f0-93ba-c64476ca29a9","71f58230-eb17-11f0-93ba-c64476ca29a9","73fd69bc-eb17-11f0-93ba-c64476ca29a9","789f2e38-eb17-11f0-93ba-c64476ca29a9","79a25dc8-eb17-11f0-93ba-c64476ca29a9","7ba98722-eb17-11f0-93ba-c64476ca29a9","7e5c6b9c-eb17-11f0-93ba-c64476ca29a9","80093d3a-eb17-11f0-93ba-c64476ca29a9","8bc179b6-eb18-11f0-93ba-c64476ca29a9","8e1003b4-eb17-11f0-93ba-c64476ca29a9","9063475c-eb17-11f0-93ba-c64476ca29a9","907ccb2c-eb18-11f0-93ba-c64476ca29a9","926b4bf8-eb17-11f0-93ba-c64476ca29a9","94c9dd92-eb17-11f0-93ba-c64476ca29a9","952056e0-eb17-11f0-93ba-c64476ca29a9","ad366292-eb17-11f0-93ba-c64476ca29a9","aee82648-eb17-11f0-93ba-c64476ca29a9","b34cdf9e-eb17-11f0-93ba-c64476ca29a9","b3f96fac-eb17-11f0-93ba-c64476ca29a9","b657ab74-eb17-11f0-93ba-c64476ca29a9","b8b985ae-eb17-11f0-93ba-c64476ca29a9","b90f0178-eb17-11f0-93ba-c64476ca29a9","b9659614-eb17-11f0-93ba-c64476ca29a9","bbc47f4c-eb17-11f0-93ba-c64476ca29a9","bc19ccb8-eb17-11f0-93ba-c64476ca29a9","bcc6ea92-eb17-11f0-93ba-c64476ca29a9","e09c5200-eb16-11f0-93ba-c64476ca29a9","e0f2124e-eb16-11f0-93ba-c64476ca29a9","e1481572-eb16-11f0-93ba-c64476ca29a9","e4ff9a1e-eb16-11f0-93ba-c64476ca29a9","e70ce4b0-eb16-11f0-93ba-c64476ca29a9","e762279a-eb16-11f0-93ba-c64476ca29a9","e80b2be2-eb16-11f0-93ba-c64476ca29a9","eb15bc62-eb16-11f0-93ba-c64476ca29a9","ee1f8988-eb16-11f0-93ba-c64476ca29a9","eeccb6d0-eb16-11f0-93ba-c64476ca29a9","f1253e34-eb16-11f0-93ba-c64476ca29a9","f37f490e-eb16-11f0-93ba-c64476ca29a9","f391611a-eb17-11f0-93ba-c64476ca29a9","f480c4ea-eb16-11f0-93ba-c64476ca29a9","f79f2922-eb17-11f0-93ba-c64476ca29a9","faa90aa2-eb17-11f0-93ba-c64476ca29a9","fe70d908-eb17-11f0-93ba-c64476ca29a9"]
total_labels = [lb for lb in os.listdir('/Applications/Deep Learning/Face Detection/data/labels')]


#Moving labels into corresponding Dataset sub-directory
for name in train_labels:
    shutil.copy(os.path.join(label_path, name), os.path.join(base_path, "train_set/labels"))

for name in validation_labels:
    shutil.copy(os.path.join(label_path, name), os.path.join(base_path, "validation_set/labels"))

#

for name in test_labels:
    shutil.copy(os.path.join(label_path, name), os.path.join(base_path, "test_set/labels"))